 <div class='bar_title'></div>



 *Introduction to Data Science*



 # Assignment 3 - Combining Data and Tidy Data Solutions



 Gunther Gust / Vanessa Haustein<br>

 Chair of Enterprise AI



 Winter Semester 24/25



 <img src='https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true' style='width:20%; float:left;' />

In [1]:
import numpy as np
import pandas as pd

 ### Exercise 1: Combining Data

 Your task will be to build a comprehensive dataset. You’ll need to use all of the datasets we’ve provided (the two customer files and the loans file) and merge them using all of the Pandas methods from the lecture.


In [2]:
# previous processing
loans = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/loans.csv')

# calculate the debt-to-income ratio
loans['debt_to_income'] = round(loans['repayment'] * 100 / loans['income'], 2)

# rename rate to interest_rate
loans.rename(columns={'rate':'interest_rate'}, inplace=True)

# calculate the total cost of the loan
loans['total_cost'] = loans['repayment'] * loans['term']

# calculate monthly profits generated
loans['profit'] = round((loans['total_cost'] * loans['interest_rate']/100)/(24), 2)

# create the risk variable
loans['risk'] = 'No'
loans.loc[loans['debt_to_income'] > 35, 'risk'] = 'Yes'

# customer profile DataFrame
customer_profile = loans.groupby('Identifier')[['repayment','debt_to_income','total_cost','profit']].sum()
customer_profile.reset_index(inplace=True)
customer_profile.head()

loans.head()

,Identifier,city,zip code,income,repayment,term,type,interest_rate,debt_to_income,total_cost,profit,risk
0,0,CHICAGO,60100,3669.0,1130.05,240,real estate,1.168,30.80,271212.0,131.99,No
1,1,DETROIT,48009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,No
2,1,DETROIT,48009,5310.0,1247.85,300,real estate,1.173,23.50,374355.0,182.97,No
3,2,SAN FRANCISCO,94010,1873.0,552.54,240,real estate,0.972,29.50,132609.6,53.71,No
4,3,SAN FRANCISCO,94010,1684.0,586.03,180,real estate,1.014,34.80,105485.4,44.57,No


 (a) First, let’s import the two customer files and look at the information they contain using `.head()` or `.info()`.

 You can find the csv files at https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/customers.csv and https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/customers_cont.csv

In [3]:
customers_1 = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/customers.csv')
customers_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   identifier  150 non-null    int64 
 1   email       150 non-null    object
 2   name        150 non-null    object
 3   gender      150 non-null    object
dtypes: int64(1), object(3)
memory usage: 4.8+ KB


In [4]:
customers_2 = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/customers_cont.csv')
customers_2.head()

,identifier,email,name,gender
0,150,EricHayes@teleworm.us,Eric Hayes,M
1,151,MonaMoreno@armyspy.com,Mona Moreno,F
2,152,VincentCraig@einrot.com,Vincent Craig,M
3,153,GlendaParsons@cuvox.de,Glenda Parsons,F
4,154,RogerWatkins@dayrep.com,Roger Watkins,M


 (b) Bring together these two DataFrames, `customers_1` and `customers_2`, into one big DataFrame called `customers` which will contain all of our customer data.

In [5]:
customers = pd.concat([customers_1, customers_2], ignore_index=True)

 (c) Now you’re going to merge the customer file with the customer profiles we created before. These profiles can be found in the `customer_profile` DataFrame. You can call this final DataFrame `data`:



 Note: There might be some customers that don't have a customer profile since their loan data is not available. Make sure they still show up in the combined dataframe in order to keep all available information.

In [6]:
data = pd.merge(customers, customer_profile, left_on='identifier', right_on='Identifier', how='outer')

 The bank’s marketing department has provided us with a file containing our customers' ages:

In [7]:
customers_age = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/customers_age.csv')
customers_age.head()

,identifier,age
0,0,54
1,1,23
2,2,3
3,3,42
4,4,47


 (d) Add the age information to the `data` DataFrame. However, it would seem that some customers who took out a loan aren’t present in this file. We need to ensure that all of the information in our `data` DataFrame is retained, so please choose your arguments with care!

In [8]:
data = pd.merge(data, customers_age, on='identifier', how='left')
data.head()

,identifier,email,name,gender,Identifier,repayment,debt_to_income,total_cost,profit,age
0,0,JohnSmith@rhyta.com,John Smith,M,0.0,1130.05,30.80,271212.0,131.99,54.0
1,1,MaryJohnson@fleckens.hu,Mary Johnson,F,1.0,1487.85,28.02,389715.0,206.66,23.0
2,2,WilliamBrown@einrot.com,William Brown,M,2.0,552.54,29.50,132609.6,53.71,3.0
3,3,JamesLee@armyspy.com,James Lee,M,3.0,586.03,34.80,105485.4,44.57,42.0
4,4,PatriciaGarcia@rhyta.com,Patricia Garcia,F,4.0,423.61,28.70,101666.4,51.21,47.0


 ### Exercise 2: Reversing melt and pivot

Why are `melt` and `pivot` not perfectly symmetric? Consider the following example and chage the code in a way that it returns the original DataFrame again.

In [9]:
stocks = pd.DataFrame({
    'year': [2015, 2015, 2016, 2016],
    'half':  [1,    2,     1,    2],
    'return': [1.88, 0.59, 0.92, 0.17]
})

(stocks.
    pivot(
    index = 'half', 
    columns = 'year', 
    values = 'return').
    melt(
    var_name = 'year', 
    value_name = 'return')
    )

,year,return
0,2015,1.88
1,2015,0.59
2,2016,0.92
3,2016,0.17


Using `pivot`, `half` becomes the index, and `year` becomes the column headers.
When `melt` is applied afterwards, it doesn't know to restore half as a regular column rather than the index.

In order to restore the original DataFrame, we need to reset the index after applying `pivot`.

In [10]:
symmetric_result = (
    stocks
    .pivot(index='half', columns='year', values='return')
    .reset_index()  # reset the index to make 'half' a column again
    .melt(id_vars='half', var_name='year', value_name='return')  # specify 'half' as an id_var
)

symmetric_result

,half,year,return
0,1,2015,1.88
1,2,2015,0.59
2,1,2016,0.92
3,2,2016,0.17


### Exercise 3: Tidy Data

The `tidyr::who` dataset contains tuberculosis (TB) cases broken down by year, country, age, gender, and diagnosis method. The data comes from the [2014 World Health Organization Global Tuberculosis Report](http://www.who.int/tb/country/data/download/en/).

 There's a wealth of epidemiological information in this dataset, but it's challenging to work with the data in the form that it's provided:

In [11]:
who = pd.read_csv("https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/who.csv")
# Fix the Namibia NA iso code.
who.loc[who.iso2.isna(),'iso2'] = "NA"
who.head()

,country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,...,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
0,Afghanistan,AF,AFG,1980,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AF,AFG,1981,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AF,AFG,1982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,AFG,1983,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AF,AFG,1984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 This is a very typical real-life example dataset. It contains redundant columns, odd variable codes, and many missing values. In short, `who` is messy, and we'll need multiple steps to tidy it. Pandas is designed so that each function does one thing well. That means in real-life situations you'll usually need to string together multiple verbs into a pipeline.



 (a) The best place to start is almost always to gather together the columns that are not variables. Have a look at the dataset and answer the following questions:



 1. Which columns are redundant?

 2. Which columns are variables?

 3. Which columns are values and not variables?

 4. What steps should be taken considering your answers from the three points above to make the dataset more tidy?

 1. It looks like `country`, `iso2`, and `iso3` are three variables that redundantly specify the country.
 2. The above and `year` are variables.
 3. We don't know what all the other columns are yet, but given the structure in the variable names (e.g. `new_sp_m014`, `new_ep_m014`, `new_ep_f014`) these are likely to be values, not variables.
4.  We need to gather together all the columns from `new_sp_m014` to `newrel_f65`. We don't know what those values represent yet, so we can give them the generic name `"key"`. We know the cells represent the count of cases, so we'll use the variable `cases`. There are a lot of missing values in the current representation, they have to be dealt with.

 (b) Considering (a), reshape the `who` data to a tidier format and drop all `NAs`. Call the new variable colum `key`.

In [12]:
who1 = who.melt(
    id_vars = ['country', 'iso2', 'iso3', 'year'], 
    var_name = 'key', 
    value_name = 'cases').dropna()
who1

,country,iso2,iso3,year,key,cases
17,Afghanistan,AF,AFG,1997,new_sp_m014,0.0
18,Afghanistan,AF,AFG,1998,new_sp_m014,30.0
19,Afghanistan,AF,AFG,1999,new_sp_m014,8.0
20,Afghanistan,AF,AFG,2000,new_sp_m014,52.0
21,Afghanistan,AF,AFG,2001,new_sp_m014,129.0
...,...,...,...,...,...,...
405269,Viet Nam,VN,VNM,2013,newrel_f65,3110.0
405303,Wallis and Futuna Islands,WF,WLF,2013,newrel_f65,2.0
405371,Yemen,YE,YEM,2013,newrel_f65,360.0
405405,Zambia,ZM,ZMB,2013,newrel_f65,669.0


(c) We can get some hint of the structure of the values in the new `key` column by counting them:

In [13]:
who1['key'].value_counts()

key
new_sp_m4554    3223
new_sp_m3544    3219
new_sp_m5564    3218
new_sp_m1524    3209
new_sp_m65      3209
new_sp_m2534    3206
new_sp_f4554    3204
new_sp_f2534    3200
new_sp_f3544    3199
new_sp_f65      3197
new_sp_f5564    3195
new_sp_f1524    3194
new_sp_f014     3174
new_sp_m014     3173
new_sn_m014     1045
new_sn_f014     1040
new_ep_m014     1038
new_ep_f014     1032
new_sn_m1524    1030
new_sn_m4554    1027
new_ep_m1524    1026
new_sn_m3544    1025
new_ep_m3544    1024
new_sn_m2534    1022
new_sn_f1524    1022
new_ep_f1524    1021
new_sn_m5564    1021
new_ep_f2534    1021
new_ep_f3544    1021
new_sn_m65      1020
new_ep_m4554    1020
new_sn_f3544    1020
new_ep_m2534    1020
new_sn_f65      1019
new_ep_m65      1018
new_sn_f4554    1018
new_ep_f4554    1017
new_sn_f5564    1017
new_ep_f5564    1017
new_sn_f2534    1016
new_ep_m5564    1015
new_ep_f65      1014
newrel_m014      190
newrel_f014      190
newrel_f65       185
newrel_m5564     185
newrel_m4554     184
newrel_m3

 (d) You might be able to parse this out by yourself with a little thought and some experimentation, but luckily we have the data dictionary handy. It tells us:



 1.  The first three letters of each column denote whether the column

     contains new or old cases of TB. In this dataset, each column contains

     new cases.



 1.  The next two letters describe the type of TB:



     *   `rel` stands for cases of relapse

     *   `ep` stands for cases of extrapulmonary TB

     *   `sn` stands for cases of pulmonary TB that could not be diagnosed by

         a pulmonary smear (smear negative)

     *   `sp` stands for cases of pulmonary TB that could be diagnosed be

         a pulmonary smear (smear positive)



 3.  The sixth letter gives the sex of TB patients. The dataset groups

     cases by males (`m`) and females (`f`).



 4.  The remaining numbers gives the age group. The dataset groups cases into

     seven age groups:



     * `014` = 0 -- 14 years old

     * `1524` = 15 -- 24 years old

     * `2534` = 25 -- 34 years old

     * `3544` = 35 -- 44 years old

     * `4554` = 45 -- 54 years old

     * `5564` = 55 -- 64 years old

     * `65` = 65 or older



 This is a lot of information in one column and we want to separate it into different columns.



 For this, we first need to make a minor fix to the format of the column names: unfortunately the names are slightly inconsistent because instead of `new_rel` we have `newrel` (it's hard to spot this here but if you don't fix it we'll get errors in subsequent steps).

In [14]:
who2 = who1.assign(names_from = lambda x: x.key.str.replace('newrel', 'new_rel'))
who2

,country,iso2,iso3,year,key,cases,names_from
17,Afghanistan,AF,AFG,1997,new_sp_m014,0.0,new_sp_m014
18,Afghanistan,AF,AFG,1998,new_sp_m014,30.0,new_sp_m014
19,Afghanistan,AF,AFG,1999,new_sp_m014,8.0,new_sp_m014
20,Afghanistan,AF,AFG,2000,new_sp_m014,52.0,new_sp_m014
21,Afghanistan,AF,AFG,2001,new_sp_m014,129.0,new_sp_m014
...,...,...,...,...,...,...,...
405269,Viet Nam,VN,VNM,2013,newrel_f65,3110.0,new_rel_f65
405303,Wallis and Futuna Islands,WF,WLF,2013,newrel_f65,2.0,new_rel_f65
405371,Yemen,YE,YEM,2013,newrel_f65,360.0,new_rel_f65
405405,Zambia,ZM,ZMB,2013,newrel_f65,669.0,new_rel_f65


 We can separate the values in each entry of the `names_from` column with `str.split()`. Create a new dataframe called `new_columns` that contains the three values in separate columns called `new`, `type` and `sexage`.

In [15]:
new_columns = (who2['names_from'].
    str.split("_",expand = True).
    rename(columns = {0: "new", 1: "type", 2: "sexage"}))
new_columns


,new,type,sexage
17,new,sp,m014
18,new,sp,m014
19,new,sp,m014
20,new,sp,m014
21,new,sp,m014
...,...,...,...
405269,new,rel,f65
405303,new,rel,f65
405371,new,rel,f65
405405,new,rel,f65


 Now, we need to separate sex and age into two different columns.

In [16]:
new_columns['sex'] = new_columns['sexage'].str[:1]
new_columns['age'] = new_columns['sexage'].str[1:]
new_columns

,new,type,sexage,sex,age
17,new,sp,m014,m,014
18,new,sp,m014,m,014
19,new,sp,m014,m,014
20,new,sp,m014,m,014
21,new,sp,m014,m,014
...,...,...,...,...,...
405269,new,rel,f65,f,65
405303,new,rel,f65,f,65
405371,new,rel,f65,f,65
405405,new,rel,f65,f,65


(e) Combine the `new_columns` dataframe with the `who` dataframe. Drop the `new` column because it's constant in this dataset. While we're dropping columns, let's also drop `iso2`, `iso3` and `sexage` since they're redundant.

In [18]:
who3 = pd.concat([
  who2.filter(items = ['country', 'year', 'cases']),
  new_columns.filter(items = ['type', 'sex', 'age'])], axis = 1)
who3

# or:
# who3 = pd.concat([
#  who2[['country', 'year']],
#  new_columns[['type', 'sex', 'age']]], axis=1)
# who3

,country,year,cases,type,sex,age
17,Afghanistan,1997,0.0,sp,m,014
18,Afghanistan,1998,30.0,sp,m,014
19,Afghanistan,1999,8.0,sp,m,014
20,Afghanistan,2000,52.0,sp,m,014
21,Afghanistan,2001,129.0,sp,m,014
...,...,...,...,...,...,...
405269,Viet Nam,2013,3110.0,rel,f,65
405303,Wallis and Futuna Islands,2013,2.0,rel,f,65
405371,Yemen,2013,360.0,rel,f,65
405405,Zambia,2013,669.0,rel,f,65


The who dataset is now ‘kinda’ tidy! We have left the age range as age for simplicity.

(f) We claimed that `iso2` and `iso3` were redundant with country. Confirm this claim.

Hint: You can use the `factorize()` method in order to encode the columns as enumerated type and then check for duplicates.

In [ ]:
who2.transform(lambda x : x.factorize()[0]).T.drop_duplicates().T

# you can see that iso2 and iso3 were removed here since they did not contain any other information than country.

,country,year,key,cases
17,0,0,0,0
18,0,1,0,1
19,0,2,0,2
20,0,3,0,3
21,0,4,0,4
...,...,...,...,...
405269,212,33,55,3621
405303,213,33,55,17
405371,215,33,55,33
405405,216,33,55,933
